In [1]:
import numpy as np 
import tensorflow as tf 
import cv2
import matplotlib.pyplot as plt

def savearray(img_array,img_name):
    cv2.imwrite(img_name,img_array)
    print('img saved:%s'%img_name)

graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
model_fn = 'tensorflow_inception_graph.pb'#导入inception模型
with tf.gfile.GFile(model_fn,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
#定义输入图像的占位符
t_input = tf.placeholder(np.float32,name='input')
#图像预处理 减均值 在训练inception模型时做了减均值处理 在此也需减同样的均值以保持一致

imagenet_mean = 117.0

#图像预处理 增加维度
#图像数据格式就是 （hight width channels）为同时将多张图片输入网络而在掐面增加一维
#变成（ batch hight width channels）

t_preprocessed = tf.expand_dims(t_input - imagenet_mean,0)
#导入模型并将预处理的图像送入网络中
tf.import_graph_def(graph_def,{'input':t_preprocessed})










#渲染函数
def render_naive(t_obj,img0,iter_n=20,step=1.0):
    #t_obj是layer_output[:,:,:,channel],即卷积层某个通道的值
    #img0 初始图像（噪声图像）
    #iter_n 迭代次数
    # step:用于控制迭代步长 可以看做学习率
    
    t_score = tf.reduce_mean(t_obj)
    #t_c=score是t_obj的平均值
    #由于我们的目标是调整输入图像使卷积层激活值尽量大
    #即最大化t_score
    #为达到此目标 可使用梯度下降
    #计算t_score对t_input的梯度
    t_grad = tf.gradients(t_score,t_input)[0]
    img = img0.copy()#复制图像可避免影响原图像的值
    for i in range(iter_n):
        #在sess中计算梯度以及当前的t_score
        g,score = sess.run([t_grad,t_score],{t_input:img})
        #对img应用梯度
        #首先对梯度进行归一化处理
        g /= g.std() + 1e-8
        #将正规化处理后的梯度应用在图像上 step 用于控制每次迭代步长 此处为1.0
        img += g*step
        #print('score(mean)=%f'%(score))
        print('iter:%d'%(i+1),'score(mean)=%f'%score)
        
        #保存图片
    savearray(img,'naive_deepddream.jpg')








#定义卷积层 通道数 并取出对应的tensor
name = 'mixed4d_3x3_bottleneck_pre_relu'
#(?,?,?,144)
#name = 'mixed4d_5x5_bottleneck_pre_relu'
#(?,?,?,32)
channel = 139
# mixed4d_3x3_bottleneck_pre_relu 共144个通道
#此处可选任意通道（0-143）之间的任意整数 进行最大化
layer_output = graph.get_tensor_by_name('import/%s:0'%name)
# 定义噪声图像
img_noise = np.random.uniform(size=(224,224,3))+100.0

#调用render_naive函数渲染
render_naive(layer_output[:,:,:,channel],img_noise,iter_n=20)
# 保存并显示图片
import PIL
im = PIL.Image.open('mountain.jpg')
im.show()
im.save('naive_single_chn.jpg')

iter:1 score(mean)=-19.989223
iter:2 score(mean)=-34.362175
iter:3 score(mean)=13.277909
iter:4 score(mean)=80.640945
iter:5 score(mean)=147.061371
iter:6 score(mean)=215.598358
iter:7 score(mean)=267.238770
iter:8 score(mean)=330.897522
iter:9 score(mean)=374.438324
iter:10 score(mean)=427.153351
iter:11 score(mean)=469.461182
iter:12 score(mean)=510.308502
iter:13 score(mean)=547.866638
iter:14 score(mean)=582.825989
iter:15 score(mean)=616.788147
iter:16 score(mean)=644.870605
iter:17 score(mean)=676.234192
iter:18 score(mean)=699.101562
iter:19 score(mean)=725.330505
iter:20 score(mean)=744.285461
img saved:naive_deepddream.jpg
